# Daisy chain generation and acquisition
This example shows how to synchronise the Red Pitaya X-channel system (daisy chain) to simultaneously acquire 16k samples of a generated signal on multiple units (fast RF inputs and outputs).

## Required hardware

- Primary Red Pitaya device (STEMlab 125-14 LN)
- One or more Secondary devices (STEMlab 125-14 LN Secondary)
- SATA cables
- SMA cables and SMA T-connectors

Connect OUT1 of the primary device to IN1 channels of all devices in the X-channel system and interconnect all devices with SATA cables.



## Marcos
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

**GENERATION**
- **Waveforms** - RP_WAVEFORM_SINE, RP_WAVEFORM_SQUARE, RP_WAVEFORM_TRIANGLE, RP_WAVEFORM_RAMP_UP, RP_WAVEFORM_RAMP_DOWN, RP_WAVEFORM_DC, RP_WAVEFORM_PWM, RP_WAVEFORM_ARBITRARY, RP_WAVEFORM_DC_NEG, RP_WAVEFORM_SWEEP
- **Generator modes** - RP_GEN_MODE_CONTINUOUS, RP_GEN_MODE_BURST
- **Sweep direction** - RP_GEN_SWEEP_DIR_NORMAL, RP_GEN_SWEEP_DIR_UP_DOWN
- **Sweep mode** - RP_GEN_SWEEP_MODE_LINEAR, RP_GEN_SWEEP_MODE_LOG
- **Generator trigger source** - RP_GEN_TRIG_SRC_INTERNAL, RP_GEN_TRIG_SRC_EXT_PE, RP_GEN_TRIG_SRC_EXT_NE
- **Generator triggers** - RP_T_CH_1, RP_T_CH_2, RP_T_CH_EXT
- **Rise and fall times** - RISE_FALL_MIN_RATIO, RISE_FALL_MAX_RATIO

**ACQUISITION**
- **Decimation** - RP_DEC_1, RP_DEC_2, RP_DEC_4, RP_DEC_8, RP_DEC_16, RP_DEC_32, RP_DEC_64, RP_DEC_128, RP_DEC_256, RP_DEC_512, RP_DEC_1024, RP_DEC_2048, RP_DEC_4096, RP_DEC_8192, RP_DEC_16384, RP_DEC_32768, RP_DEC_65536 
- **Acquisition trigger** - RP_TRIG_SRC_DISABLED, RP_TRIG_SRC_NOW, RP_TRIG_SRC_CHA_PE, RP_TRIG_SRC_CHA_NE, RP_TRIG_SRC_CHB_PE, RP_TRIG_SRC_CHB_NE, RP_TRIG_SRC_EXT_PE, RP_TRIG_SRC_EXT_NE, RP_TRIG_SRC_AWG_PE, RP_TRIG_SRC_AWG_NE
- **Acquisition trigger state** - RP_TRIG_STATE_TRIGGERED, RP_TRIG_STATE_WAITING
- **Buffer size** - ADC_BUFFER_SIZE, DAC_BUFFER_SIZE
- **Fast analog gen_channels** - RP_CH_1, RP_CH_2

**MULTICHANNEL**
- **Shared trigger source** - OUT_TR_ADC, OUT_TR_DAC

SIGNALlab 250-12 only:
- **Input coupling** - RP_DC, RP_AC
- **Generator gain** - RP_GAIN_1X, RP_GAIN_5X

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Acquisition trigger** - RP_TRIG_SRC_CHC_PE, RP_TRIG_SRC_CHC_NE, RP_TRIG_SRC_CHD_PE, RP_TRIG_SRC_CHD_NE

## X-channel system
Red Pitaya X-channel system consists of one primary device that shares its *CLK* and *TRIG* signals to all other secondary devices. For more information on X-channel system, please reffer to the [X-channel documentation](https://redpitaya.readthedocs.io/en/latest/developerGuide/hardware/125-14_MULTI/top.html#stemlab-125-14-x-channel-system).

The example itself is similar to what we have done in the Generation and Acquisition examples, but it has to be run on multiple boards at the same time.

Here is a short briefing on the new functions used:
- **rp_SetEnableDaisyChainTrigSync(bool_state)** - Set trigger synchronisation.
- **rp_GetEnableDaisyChainTrigSync()** - Get trigger synchronisation
- **rp_SetEnableDiasyChainClockSync(bool_state)** - Set clock synchronisation.
- **rp_GetEnableDiasyChainClockSync()** - Get clock synchronisation
- **rp_SetSourceTrigOutput(shared_trigger_source)** - Choose whether ADC or DAC trigger is synchronised between units.
- **rp_GetSourceTrigOutput()** - Check whether ADC or DAC trigger is synchronised between units.

With these functions we are able to synchronise the *clock* and *trigger* signals between all boards.

### Note
The codes for primary and secondary units differ slightly.


### Primary unit code
The primary unit is responsible for sharing its CLK and TRIG signals to all other units through the SATA extension connectors.

Parameters:

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()

########! Primary unit code !#########
gen_channel = rp.RP_CH_1        # rp.RP_CH_2
waveform = rp.RP_WAVEFORM_SINE
freq = 100000
ampl = 1.0

gen_trig_sour = rp.RP_GEN_TRIG_SRC_INTERNAL

trig_lvl = 0.5
trig_dly = 0

acq_channel = rp.RP_CH_1
acq_trigger_channel = rp.RP_T_CH_1
dec = rp.RP_DEC_1

acq_trig_sour = rp.RP_TRIG_SRC_CHA_PE

N = 16384

# Initialize the interface
rp.rp_Init()

# Reset Generation and Acquisition
rp.rp_GenReset()
rp.rp_AcqReset()

Enabling Daisy chain:

In [ ]:
rp.rp_SetEnableDiasyChainClockSync(True)        # Sync Clock
rp.rp_SetEnableDaisyChainTrigSync(True)         # Sync Trigger

# Choose which trigger to synchronise (rp.OUT_TR_ADC, rp.OUT_TR_DAC)
rp.rp_SetSourceTrigOutput(rp.OUT_TR_ADC)

# LED indicator
rp.rp_DpinSetState(rp.RP_LED5, rp.RP_HIGH)

Generation code on OUT1 of primary unit:

In [ ]:
print("Gen_start")
rp.rp_GenWaveform(gen_channel, waveform)
rp.rp_GenFreqDirect(gen_channel, freq)
rp.rp_GenAmp(gen_channel, ampl)

rp.rp_GenTriggerSource(gen_channel, gen_trig_sour)
rp.rp_GenOutEnable(gen_channel)

Acquisition code is the same as on a normal unit:

In [ ]:
rp.rp_AcqSetDecimation(dec)

# Set trigger level and delay
rp.rp_AcqSetTriggerLevel(acq_trigger_channel, trig_lvl)
rp.rp_AcqSetTriggerDelay(trig_dly)

# Start Acquisition
print("Acq_start")
rp.rp_AcqStart()

# Specify trigger - input 1 positive edge
rp.rp_AcqSetTriggerSrc(acq_trig_sour)

time.sleep(1)

rp.rp_GenTriggerOnly(gen_channel)       # Trigger generator

# Trigger state
while 1:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        break

# Fill state
while 1:
    if rp.rp_AcqGetBufferFillState()[1]:
        break

### Get data ###
# Volts
fbuff = rp.fBuffer(N)
res = rp.rp_AcqGetOldestDataV(acq_channel, N, fbuff)[1]

data_V = np.zeros(N, dtype = float)

for i in range(0, N, 1):
    data_V[i] = fbuff[i]

plt.plot(data_V)
plt.show()


# Release resources
rp.rp_Release()

### Secondary unit code
The major difference between primary and secondary unit codes is that we do not need the generator but must specify the acquisition trigger as **EXT_NE**.

The clock and trigger signals pass through every secondary unit's FPGA.

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()

########! Secondary unit code !#########

acq_channel = rp.RP_CH_1        # rp.RP_CH_2
trig_lvl = 0.5
trig_dly = 0

dec = rp.RP_DEC_1

rp.rp_Init()

# Reset Acquisition
rp.rp_AcqReset()

###### Enable Daisy Chain #####
rp.rp_SetEnableDiasyChainClockSync(True)        # Sync Clock
rp.rp_SetEnableDaisyChainTrigSync(True)         # Sync Trigger

# Choose which trigger to synchronise (rp.OUT_TR_ADC, rp.OUT_TR_DAC)
rp.rp_SetSourceTrigOutput(rp.OUT_TR_ADC)

# LED indicator
rp.rp_DpinSetState(rp.RP_LED5, rp.RP_HIGH)


##### Acquisition #####
rp.rp_AcqSetDecimation(dec)
rp.rp_AcqSetTriggerDelay(trig_dly)

# Start Acquisition
print("Acq_start")
rp.rp_AcqStart()

#!!! Specify trigger - must be EXT_NE !!!#
rp.rp_AcqSetTriggerSrc(rp.RP_TRIG_SRC_EXT_NE)

# Trigger state
while 1:
    trig_state = rp.rp_AcqGetTriggerState()[1]
    if trig_state == rp.RP_TRIG_STATE_TRIGGERED:
        break

# Fill state
while 1:
    if rp.rp_AcqGetBufferFillState()[1]:
        break

### Get data ###
# Volts
fbuff = rp.fBuffer(N)
res = rp.rp_AcqGetOldestDataV(rp.RP_CH_1, N, fbuff)[1]
data_V = np.zeros(N, dtype = float)

for i in range(0, N, 1):
    data_V[i] = fbuff[i]

plt.plot(data_V)
plt.show()


# Release resources
rp.rp_Release()

### Note
There are a lot of different commands for the acquisition and generation. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [Command table](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/command_list.html#list-of-supported-scpi-api-commands) for all available commands.
